In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("chapter7_analytical_data") \
      .config("spark.executor.memory", '3g') \
      .config("spark.executor.cores", '3') \
      .config("spark.cores.max", '3') \
      .getOrCreate()


23/01/18 21:02:21 WARN Utils: Your hostname, DESKTOP-DVUDB98 resolves to a loopback address: 127.0.1.1; using 172.26.177.75 instead (on interface eth0)
23/01/18 21:02:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/01/18 21:02:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.parquet('chapter7_parquet_files/yellow_tripdata_2022-01.parquet')

In [9]:
df.toPandas().head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.00,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.00,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.00,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.00,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.00,0.3,30.30,2.5,0.0
5,1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.30,1.0,N,138,161,1,33.0,3.0,0.5,13.00,6.55,0.3,56.35,2.5,0.0
6,2,2022-01-01 00:20:50,2022-01-01 00:34:58,1.0,5.07,1.0,N,233,87,1,17.0,0.5,0.5,5.20,0.00,0.3,26.00,2.5,0.0
7,2,2022-01-01 00:13:04,2022-01-01 00:22:45,1.0,2.02,1.0,N,238,152,2,9.0,0.5,0.5,0.00,0.00,0.3,12.80,2.5,0.0
8,2,2022-01-01 00:30:02,2022-01-01 00:44:49,1.0,2.71,1.0,N,166,236,1,12.0,0.5,0.5,2.25,0.00,0.3,18.05,2.5,0.0
9,2,2022-01-01 00:48:52,2022-01-01 00:53:28,1.0,0.78,1.0,N,236,141,2,5.0,0.5,0.5,0.00,0.00,0.3,8.80,2.5,0.0


In [19]:
from pyspark.sql.functions import year, month, dayofmonth

df_month = df.withColumn("month", month("tpep_pickup_datetime"))

In [20]:
df_month.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|month|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----+
|       1| 2022-01-01 00:35:40|  2022-01-01 00:53:29|            2.0|          3.8|       1.0|                 N|         142|         236|           1|       14.5|  3.0

In [22]:
df_month.write.mode('overwrite').partitionBy("month").parquet('chapter7_partitioned_files/')

In [7]:
df.write.format('avro').save('chapter7_parquet_files/file.avro')

In [2]:
from pyspark.sql import SparkSession
spark2 = SparkSession.builder \
      .master("local[1]") \
      .appName("chapter7_analytical_data2") \
      .config("spark.executor.memory", '3g') \
      .config("spark.executor.cores", '2') \
      .config("spark.cores.max", '2') \
      .config("spark.jars.packages", 'org.apache.spark:spark-avro_2.12:3.3.1') \
      .getOrCreate()

23/01/18 20:32:26 WARN Utils: Your hostname, DESKTOP-DVUDB98 resolves to a loopback address: 127.0.1.1; using 172.26.177.75 instead (on interface eth0)
23/01/18 20:32:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/glauesppen/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/glauesppen/.ivy2/cache
The jars for the packages stored in: /home/glauesppen/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e4b76bcb-3222-47cf-ba8c-87fbbac41048;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.3.1 in central
	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 110ms :: artifacts dl 2ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.3.1 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	org.tukaani#xz;1.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      defau

In [27]:
spark2.stop()

In [29]:
spark2

In [10]:
df = spark.read.format('avro').load('chapter7_parquet_files/file.avro')

In [13]:
df.show()

+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        actionType|   mechanismOfAction|           chemblIds|          targetName|          targetType|             targets|          references|
+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| HYDROLYTIC ENZYME|Hyaluronic acid h...|     [CHEMBL1201718]|                null|                null|                null|[{FDA, [label/201...|
|  DISRUPTING AGENT|Cell membrane dis...|     [CHEMBL1187417]|                null|                null|                null|[{Other, [https:/...|
|         INHIBITOR|Norepinephrine tr...|[CHEMBL1722, CHEM...|Norepinephrine tr...|      single protein|   [ENSG00000103546]|[{PubMed, [232848...|
|         INHIBITOR|Beta amyloid A4 p...|     [CHEMBL1743004]|Beta amyloid A4 p...|      single protein|   [ENSG000001

In [16]:
df.printSchema()

root
 |-- actionType: string (nullable = true)
 |-- mechanismOfAction: string (nullable = true)
 |-- chemblIds: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- targetName: string (nullable = true)
 |-- targetType: string (nullable = true)
 |-- targets: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- references: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- source: string (nullable = true)
 |    |    |-- ids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- urls: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)



In [25]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [38]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, DateType
schema = StructType([ \
    StructField("vendorId", LongType() ,True), \
    StructField("tpep_pickup_datetime", DateType() ,True), \
    StructField("tpep_dropoff_datetime", DateType() ,True), \
    StructField("passenger_count", DoubleType() ,True), \
    StructField("trip_distance", DoubleType() ,True), \
    StructField("ratecodeId", DoubleType() ,True), \
    StructField("store_and_fwd_flag", StringType() ,True), \
    StructField("puLocationId", LongType() ,True), \
    StructField("doLocationId", LongType() ,True), \
    StructField("payment_type", LongType() ,True), \
    StructField("fare_amount", DoubleType() ,True), \
    StructField("extra", DoubleType() ,True), \
    StructField("mta_tax", DoubleType() ,True), \
    StructField("tip_amount", DoubleType() ,True), \
    StructField("tolls_amount", DoubleType() ,True), \
    StructField("improvement_surcharge", DoubleType() ,True), \
    StructField("total_amount", DoubleType() ,True), \
    StructField("congestion_surcharge", DoubleType() ,True), \
    StructField("airport_fee", DoubleType() ,True), \
])


In [39]:
df_new_schema = spark.read.schema(schema).parquet('chapter7_parquet_files/yellow_tripdata_2022-01.parquet')

In [43]:
df_new_schema.toPandas().head(10)

,vendorId,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeId,store_and_fwd_flag,puLocationId,doLocationId,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.00,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.00,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.00,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.00,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.00,0.3,30.30,2.5,0.0
5,1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.30,1.0,N,138,161,1,33.0,3.0,0.5,13.00,6.55,0.3,56.35,2.5,0.0
6,2,2022-01-01 00:20:50,2022-01-01 00:34:58,1.0,5.07,1.0,N,233,87,1,17.0,0.5,0.5,5.20,0.00,0.3,26.00,2.5,0.0
7,2,2022-01-01 00:13:04,2022-01-01 00:22:45,1.0,2.02,1.0,N,238,152,2,9.0,0.5,0.5,0.00,0.00,0.3,12.80,2.5,0.0
8,2,2022-01-01 00:30:02,2022-01-01 00:44:49,1.0,2.71,1.0,N,166,236,1,12.0,0.5,0.5,2.25,0.00,0.3,18.05,2.5,0.0
9,2,2022-01-01 00:48:52,2022-01-01 00:53:28,1.0,0.78,1.0,N,236,141,2,5.0,0.5,0.5,0.00,0.00,0.3,8.80,2.5,0.0


In [53]:
df.groupBy("vendorId").count().orderBy("vendorId").show()

+--------+-------+
|vendorId|  count|
+--------+-------+
|       1| 742273|
|       2|1716059|
|       5|     36|
|       6|   5563|
+--------+-------+



In [72]:
from pyspark.sql.functions import year, month, dayofmonth, hour
df.groupBy(month("tpep_pickup_datetime")).count().orderBy("count").show()

+---------------------------+-------+
|month(tpep_pickup_datetime)|  count|
+---------------------------+-------+
|                          4|      1|
|                          5|      2|
|                          3|      4|
|                          2|      7|
|                         12|     30|
|                          1|2463887|
+---------------------------+-------+



In [62]:
df.createOrReplaceTempView("ny_yellow_taxi_data")

In [73]:
select = spark.sql(
"""
SELECT * FROM ny_yellow_taxi_data
GROUP BY vendorId
"""
)

AnalysisException: expression 'ny_yellow_taxi_data.`tpep_pickup_datetime`' is neither present in the group by, nor is it an aggregate function. Add to group by or wrap in first() (or first_value) if you don't care which value you get.;
Aggregate [vendorId#0L], [VendorID#0L, tpep_pickup_datetime#1, tpep_dropoff_datetime#2, passenger_count#3, trip_distance#4, RatecodeID#5, store_and_fwd_flag#6, PULocationID#7L, DOLocationID#8L, payment_type#9L, fare_amount#10, extra#11, mta_tax#12, tip_amount#13, tolls_amount#14, improvement_surcharge#15, total_amount#16, congestion_surcharge#17, airport_fee#18]
+- SubqueryAlias ny_yellow_taxi_data
   +- Relation[VendorID#0L,tpep_pickup_datetime#1,tpep_dropoff_datetime#2,passenger_count#3,trip_distance#4,RatecodeID#5,store_and_fwd_flag#6,PULocationID#7L,DOLocationID#8L,payment_type#9L,fare_amount#10,extra#11,mta_tax#12,tip_amount#13,tolls_amount#14,improvement_surcharge#15,total_amount#16,congestion_surcharge#17,airport_fee#18] parquet


In [71]:
select.show()

+--------+--------+
|vendorId|count(1)|
+--------+--------+
|       5|      36|
|       6|    5563|
|       1|  742273|
|       2| 1716059|
+--------+--------+



In [79]:
df_partitioned = spark.read.parquet("chapter7_partitioned_files/month=5/")

In [80]:
df_partitioned.toPandas()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2022-05-18 21:17:19,2022-05-18 21:22:58,4.0,1.47,1.0,N,48,142,1,6.5,1.0,0.5,2.16,0.0,0.3,12.96,2.5,0.0
1,2,2022-05-18 21:41:57,2022-05-18 21:47:45,4.0,1.35,1.0,N,50,142,1,6.5,1.0,0.5,1.00,0.0,0.3,11.80,2.5,0.0
